In [7]:
import numpy as np
import os
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib import animation
import pickle
from itertools import combinations
from scipy import stats

Read in the decoding score data

In [3]:
#read in decoding accuracy scores

#set up directory to read in decoding scores
S = 'S03'
rootdir = os.getcwd()
outdir = rootdir + '/plots'
in_filepath = rootdir + '/' + S + '_IMbased_allscores.pkl'

#load scores from the pickle file into all_scores, a dictionary of the following format: 
# key: tuple of two trial codes representing a pair of images ex. (1,2)
# value: a 1x1081 list of decoding scores at each timepoint
in_file = open(in_filepath,'rb')
all_scores = pickle.load(in_file)
in_file.close()

#all_scores has 7750 entries, representing every pair of images 1-125
print(len(all_scores))

7750


Build the MEG RDM

In [49]:
#convert the all_scores dictionary to a 3D numpy array (MEG RDM)
#only fills in the upper triangle, off diagonal

nrows = 125
nsamples = len(all_scores[1,2])

#initialize a numpy array of zeros, size: 125x125x1081
rdm_meg = np.zeros((nrows,nrows,nsamples))

#iterate through each entry in all_scores
for key,value in all_scores.items():

    #each key dictates the row and column number we want
    row = key[0] - 1
    col = key[1] - 1

    #fill in the corresponding list of decoding accuracies in symmetric positions in the array
    rdm_meg[row,col] = value
    rdm_meg[col,row] = value

print(rdm_meg[0,124,0])


#plot a single snapshot of the MEG RDM
plt.imshow(rdm_meg[:,:,540])
plt.title('MEG RDM at ~350 ms')
plt.colorbar()



0.65


plot the MEG RDM 

In [44]:
#Use animation feature to visualize RDM over time
fig = plt.figure()

def animate(i):
    plt.imshow(rdm_meg[:,:,i])

anim = animation.FuncAnimation(fig,animate,interval=5)

Build Model RDMs based on trial structure

In [6]:
#these three helper functions generate trial codes for the (5 actors x 5 expr x 5 views)
#input: number 1-5 representing actor, expression or view code
#output: list of all trial codes correspoding to that actor, expression or view

#these functions are used as arguments (trialcode_func) for later functions build_trial_key and build_model_rdm

def get_trials_by_expression(exp_code):
    trial_codes = []
    start_code = (exp_code*5) - 4
    
    for i in range(start_code,125,25):
        for j in range(i,i+5):
            trial_codes.append(j)
    return trial_codes

def get_trials_by_id(actor_code):
    start_code = (actor_code * 25) - 24
    return [i for i in range(start_code,start_code+25)]

def get_trials_by_view(view_code):
    return [i for i in range(view_code, 126,5)]


#helper function for building model rdms
def build_trial_key(trialcode_func):
    #takes in one of the three functions above and returns a dict mapping each trial code to its corresponding id, expression, or view code
    key = {}
    for i in range(5):
        codes = (trialcode_func(i+1))
        #print(codes)
        
        for code in codes:
            key[code] = i+1

    return key

print(build_trial_key(get_trials_by_view))

{1: 1, 6: 1, 11: 1, 16: 1, 21: 1, 26: 1, 31: 1, 36: 1, 41: 1, 46: 1, 51: 1, 56: 1, 61: 1, 66: 1, 71: 1, 76: 1, 81: 1, 86: 1, 91: 1, 96: 1, 101: 1, 106: 1, 111: 1, 116: 1, 121: 1, 2: 2, 7: 2, 12: 2, 17: 2, 22: 2, 27: 2, 32: 2, 37: 2, 42: 2, 47: 2, 52: 2, 57: 2, 62: 2, 67: 2, 72: 2, 77: 2, 82: 2, 87: 2, 92: 2, 97: 2, 102: 2, 107: 2, 112: 2, 117: 2, 122: 2, 3: 3, 8: 3, 13: 3, 18: 3, 23: 3, 28: 3, 33: 3, 38: 3, 43: 3, 48: 3, 53: 3, 58: 3, 63: 3, 68: 3, 73: 3, 78: 3, 83: 3, 88: 3, 93: 3, 98: 3, 103: 3, 108: 3, 113: 3, 118: 3, 123: 3, 4: 4, 9: 4, 14: 4, 19: 4, 24: 4, 29: 4, 34: 4, 39: 4, 44: 4, 49: 4, 54: 4, 59: 4, 64: 4, 69: 4, 74: 4, 79: 4, 84: 4, 89: 4, 94: 4, 99: 4, 104: 4, 109: 4, 114: 4, 119: 4, 124: 4, 5: 5, 10: 5, 15: 5, 20: 5, 25: 5, 30: 5, 35: 5, 40: 5, 45: 5, 50: 5, 55: 5, 60: 5, 65: 5, 70: 5, 75: 5, 80: 5, 85: 5, 90: 5, 95: 5, 100: 5, 105: 5, 110: 5, 115: 5, 120: 5, 125: 5}


In [14]:
#build a model rdm for each condition

def build_model_rdm(trialcode_func):

    #build the key linking trial codes to exp/id/view codes
    key = build_trial_key(trialcode_func)

    n = len(key)

    #initialize an empty numpy array size 125x125
    rdm = np.full((n,n),np.nan)

    #iterate through each row and column number to build the model rdm
    for r in range(n):

        for c in range(n):

            #if the two trial codes map to the same expression, id or view, fill this entry with 0
            if key[r+1] == key[c+1]:
                rdm[r,c] = 0
            #if different, fill this entry with a value of 1
            else:
                rdm[r,c] = 1

    return rdm

rdm_view = build_model_rdm(get_trials_by_view)
rdm_id = build_model_rdm(get_trials_by_id)
rdm_exp = build_model_rdm(get_trials_by_expression)


In [57]:
#plot the 3 model RDM as subplots on a single figure
fig,ax = plt.subplots(1,3)

ax[0].imshow(rdm_id)
ax[0].set_title('Identity')
ax[1].imshow(rdm_exp)
ax[1].set_title('Expression')
ax[2].imshow(rdm_view)
ax[2].set_title('Viewpoint')

#adjust the spacing between subplots to avoid overlap
fig.tight_layout()

Compute similarity

In [40]:
#function to correlate one model rdm with every timepoint of the meg rdm
#return a 1x1081 list: timecourse of Spearman correlations
def rsa(model, meg):

    timecourse = []

    #extract the upper triangle from the model rdm and flatten it, mask values of 0 so they aren't included in the correlation
    model_flat = ma.masked_equal(np.ndarray.flatten(np.triu(model,k=1)),0)

    #iterate through each timepoint
    for i in range(len(meg[0,0])): # 

        #extract the meg rdm for that timepoint
        slice = meg[:,:,i]

        #extract the upper triangle and flatten the meg rdm
        slice_flat = ma.masked_equal(np.ndarray.flatten(np.triu(slice,k=1)),0)

        #compute Spearman's R between the model and meg 
        #not keeping track of p right now but can be changed
        correlation,p = stats.spearmanr(model_flat,slice_flat)

        timecourse.append(correlation)

    return timecourse


rsa_view = rsa(rdm_view,rdm_meg)
rsa_id = rsa(rdm_id,rdm_meg)
rsa_exp = rsa(rdm_exp,rdm_meg)
print(len(rsa_id))



1081


In [62]:
#plot results of rsa
#set ranges for x and y axes
xmin = -0.1
xmax = 0.8
ymin = 0.75
ymax = 0.85
axes = [xmin,xmax,ymin,ymax]

#generate evenly spaced values for the x-axis
nsamples = len(rsa_view)
xvals = np.linspace(xmin,xmax,nsamples)


plt.plot(xvals,rsa_exp,label='expression')
plt.plot(xvals,rsa_id,label='identity')
plt.plot(xvals,rsa_view,label='viewpoint')
plt.axis(axes)
plt.title('Timecourse of Spearman\'s correlations from RSA')
plt.xlabel('time (s)')
plt.ylabel('Spearman\'s R')
plt.legend()